# NBA twitter for mining and NLP learning

work in progress, based loosesly off of https://github.com/elaiken3/twitter_api-nlp-project1/blob/master/twitter_api-nlp-project.ipynb


In [11]:
%matplotlib inline
import config_keys
import twitter
import re 
import datetime
import pandas as pd
import numpy as np

twitter_api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode = 'extended'
)

In [12]:
#TweetMiner function from Mike Roman

class TweetMiner(object):

    def __init__(self, api, result_limit = 20):
        
        self.api = api        
        self.result_limit = result_limit
        
    def mine_user_tweets(self, user="HillaryClinton", mine_retweets=False, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [115]:
#tweets_embiid = api.GetUserTimeline(screen_name="JoelEmbiid", count=20, include_rts=False)
#tweets_embiid = [x.AsDict() for x in tweets_embiid]
#for tweet in tweets_embiid:
#    print tweet['id']
#    print tweet['full_text']
#    print'\n'
    
# Result limit == count parameter from our GetUserTimeline()
miner = TweetMiner(twitter_api, result_limit=200)

In [232]:
#embiid = miner.mine_user_tweets(user="JoelEmbiid")
james = miner.mine_user_tweets(user="KingJames")
mccollum = miner.mine_user_tweets(user="CJMcCollum")
durant = miner.mine_user_tweets(user="KDTrey5")
#lillard = miner.mine_user_tweets(user="Dame_Lillard")
woj = miner.mine_user_tweets(user="wojespn")
shams = miner.mine_user_tweets(user="shamscharania")

In [138]:
#def remove_non_ascii(text):
#    ''.join(i for i in text if ord(i)<128)

In [244]:
#for i in range(5):
#    print embiid[i]['text']
#    print(' ')
    
#embiid_df = pd.DataFrame(embiid)
james_df = pd.DataFrame(james)
mccollum_df= pd.DataFrame(mccollum)
durant_df = pd.DataFrame(durant)
#lillard_df = pd.DataFrame(lillard)
woj_df= pd.DataFrame(woj)
shams_df = pd.DataFrame(shams)

#tweets = pd.concat([embiid_df, james_df, mccollum_df,durant_df,lillard_df], axis=0)

#tweets = pd.concat([embiid_df,lillard_df, james_df], axis=0)
tweets = pd.concat([woj_df,durant_df], axis=0)


#tweets['text'] = tweets['text'].apply(remove_non_ascii)
#tweets['text'] = tweets['text'].astype(str)
 
tweets.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Sun Feb 17 21:23:20 +0000 2019,wojespn,2019-02-18 13:41:48.062467,112,“The more swings you get the more chances you ...,1097245133622382592
1,Sun Feb 17 17:23:27 +0000 2019,wojespn,2019-02-18 13:41:48.062488,258,Atlanta Hawks GM Travis Schlenk on the franchi...,1097184766347145216
2,Fri Feb 15 23:12:36 +0000 2019,wojespn,2019-02-18 13:41:48.062493,84,ESPN story on Tim Connelly’s contract extensio...,1096547855085592576
3,Fri Feb 15 22:55:05 +0000 2019,wojespn,2019-02-18 13:41:48.062498,117,Connelly’s contract could’ve expired at end of...,1096543448868573184
4,Fri Feb 15 22:54:14 +0000 2019,wojespn,2019-02-18 13:41:48.062502,522,Denver Nuggets president of basketball operati...,1096543232572444672


In [245]:
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(t, fix_unicode=True, lowercase=True, 
                              no_urls=True, no_emails=True, no_phone_numbers=True,
                              no_currency_symbols=True, no_punct=True, no_accents=True)
              for t in tweet_text]

print tweet_text[1:8]
print '-------------------------------------------------------------------------------------'
print clean_text[1:8]


[u"Atlanta Hawks GM Travis Schlenk on the franchise's rebuild, Trae Young/Luka Doncic draft night trade, his years at the ground floor of Warriors dynasty, working in Pat Riley's film room and much more. The Woj Pod: https://t.co/DK66e9yPpf"
 u'ESPN story on Tim Connelly\u2019s contract extension with the revitalized Denver Nuggets. https://t.co/UnEQgHGlV1'
 u'Connelly\u2019s contract could\u2019ve expired at end of this season, but discussions with Nuggets president Josh Kroenke had been ongoing in recent weeks and culminated with a new deal, league sources said. https://t.co/1V8RgCfFCi'
 u'Denver Nuggets president of basketball operations Tim Connelly, architect of one of the NBA\u2019s most impressive recent rebuilds, has agreed to a contract extension, league sources tell ESPN.'
 u'Ferry has worked with the Pels in a part-time capacity as a consultant, but his most recent work as GM for the Atlanta Hawks was viewed league-wide as a clinic on how to reshape a roster and franchise wi

In [231]:

#def remove_pattern(input_txt, pattern):
#    r = re.findall(pattern, input_txt)
#    for i in r:
#        input_txt = re.sub(i, '', input_txt)
#        
#    return input_txt    

#tweets['text'] = np.vectorize(remove_pattern)(tweets['text'], "@[\w]*")
#tweets['text'] = tweets['text'].str.replace("[^a-zA-Z#]", " ")
#tweets['text'] = tweets['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
#tweets['text'] = tweets['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])


#tokenized_tweet = tweets['text'].apply(lambda x: x.split())
#tokenized_tweet.head()

0    [From, name, Joel, Hulu, Live, Sports, Embiid,...
1    [Stars, live, Recruit, your, Star, team, #NBAL...
2                                      [https, CCpzig]
3    [Lolololololololololololololololololololololol...
4    [honor, part, stringers, against, real, starte...
Name: text, dtype: object

In [254]:
y = tweets['handle'].map(lambda x: 1 if x == 'wojespn' else 0).values
print max(pd.Series(y).value_counts(normalize=True))

0.531705539359


In [255]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


#Vectorizing with TF-IDF Vectorizer and creating X matrix
tfv = TfidfVectorizer(ngram_range=(2,4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()
print X.shape

(5488, 2000)


In [256]:
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression()
LogisticRegression(solver='lbfgs')

params = {'penalty': ['l1', 'l2'], 'C':np.logspace(-5,0,100)}
#Grid searching to find optimal parameters for Logistic Regression
gs = GridSearchCV(lr, param_grid=params, cv=10, verbose=1)
gs.fit(X, y)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:  2.8min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([1.00000e-05, 1.12332e-05, ..., 8.90215e-01, 1.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [257]:
print gs.best_params_
print gs.best_score_

{'penalty': 'l2', 'C': 1.0}
0.9092565597667639


In [258]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)

print accuracies.mean()
print 1-y.mean()

0.9092367675800727
0.5317055393586005


In [259]:
estimator = LogisticRegression(penalty='l2',C=1.0)
estimator.fit(X,y)

# Prep our source as TfIdf vectors
source_test = [
    "That’s was AMAZING!!! Got me so hyped I wanted to be in it too! Kudos",
    "ESPN story on Tim Connelly’s contract extension with the revitalized Denver Nuggets."
]
Xtest = tfv.transform(source_test)
pd.DataFrame(estimator.predict_proba(Xtest), columns=["Proba_durant", "Proba_woj"])

,Proba_durant,Proba_woj
0,0.720873,0.279127
1,0.037079,0.962921


In [ ]:




from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

#cv = CountVectorizer()

for i in range(5):
    print tweets[i]['text']
    print(' ')
    
#tweets['text'] = tweets['text'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))

#list_tweets =tweets['text'].tolist()
#[x.encode('UTF8') for x in list_tweets]


#vectorizer = TfidfVectorizer(ngram_range=(2,4), stop_words='english',sublinear_tf=True)



#count_vec = CountVectorizer(stop_words="english", analyzer='word', 
#                            ngram_range=(2, 4), max_df=1.0, min_df=1, max_features=200)

#count_train = count_vec.fit(list_tweets)
#bag_of_words = count_vec.transform(list_tweets)
#print("\nEvery 3rd feature:\n{}".format(count_vec.get_feature_names()))

#print("Vocabulary size: {}".format(len(count_train.vocabulary_)))



#print list_tweets

#X = vectorizer.fit_transform(list_tweets)
#print(vectorizer.get_feature_names())

#print(X.shape)


#summaries = "".join(tweets['text'])
#X_train_counts = cv.fit_transform(embiid_df.text)
#X_train_counts.shape
#cv.vocabulary_

#print summaries

#ngrams_summaries = vectorizer.build_analyzer()(X)

#Counter(ngrams_summaries).most_common(30)

In [105]:
print("Vocabulary size: {}".format(len(count_train.vocabulary_)))
print count_train.vocabulary_


Vocabulary size: 200
{u'thanks lot': 167, u'best luck': 8, u'kingjames favorite': 94, u'stay tuned': 157, u'girls ain loyal': 54, u'lt lt lt lt': 123, u'wearefamily https': 185, u'lot chronicles': 115, u'hard work': 81, u'don worry': 43, u'great time': 67, u'facts https': 45, u'just like': 90, u'working hard': 191, u'lol just': 109, u'good game': 58, u'allez les': 3, u'boss https': 14, u'love https': 118, u'damedolla frontpagemusic': 31, u'really appreciate': 142, u'ripcitygoon simzadam_jr': 146, u'hard kind': 80, u'let going': 99, u'good day': 57, u'time https': 174, u'gonna miss': 56, u'happy new': 78, u'parking lot chronicles': 135, u'happy thanksgiving': 79, u'new pulluppod': 132, u'world https': 192, u'joel embiid nbavote': 87, u'appreciate love': 7, u'love thanks': 119, u'album confirmed': 2, u'day https': 35, u'lil homie': 104, u'good morning': 61, u'big time': 12, u'good time': 62, u'look like': 113, u'good win': 63, u'certainly https': 21, u'month https': 129, u'super bowl': 1